## imports

In [169]:
import numpy as np
import math
from tqdm import tqdm, trange

## functions

convert an integer $x$ to a zero-padded binary string with $d$ digits:

In [170]:
def to_bin(x, d):
    assert d >= 0
    
    if d == 0:
        return ""
    else:
        return f'{x:0{d}b}'

get the $n$th triangular number:

In [171]:
def T(n):
    return int((n+1) * n / 2)

check if next node in graph represented by `G_str` can be added to independent set represented by `selected_str`:

In [172]:
def conflicts(G_str, selected_str):
    d = len(selected_str)
    
    if d == 0:
        return False

    to_check = G_str[T(d-1):T(d-1)+d]
    for x, y in zip(selected_str, to_check):
        if x == "1" and y == "1":
            return True

get the edge corresponding to the $j$th character in a `G_str`:

In [173]:
def find_edge(j):
    assert j >= 0
    
    temp = 0
    i = 1
    while temp + i < j + 1:
        temp += i
        i += 1
    return j - temp, i

print the edges of the graph represented by the string `G_str`:

In [183]:
def print_graph(G_str):
    for j, e in enumerate(G_str):
        if e == "1":
            x, y = find_edge(j)
            print((x+1, y+1))

compute competitive ratio for $n$ nodes and probability $p$:

In [174]:
def compute_comp_ratio(n, p):
    num_pairs = T(n-1)
    
    first_iter = True
    for G_int in range(2**num_pairs):
        G_str = to_bin(G_int, num_pairs)

        # optimal offline algorithm
        # note: this could be marginally sped up by going in decreasing order of selected set size and breaking once a valid selection is found
        opt_val = 0
        for i in range(2**n):
            selected_str = to_bin(i, n)
            valid_sel = True
            for j, e in enumerate(G_str):
                if e == "1":
                    x, y = find_edge(j)
                    if selected_str[x] == "1" and selected_str[y] == "1":
                        valid_sel = False
                        break
            if valid_sel:
                val = 0
                for node in selected_str:
                    if node == "1":
                        val += 1
                opt_val = max(opt_val, val)


        # probabilistic greedy algorithm
        F = {}

        for i in range(2**n):
            F[to_bin(i, n)] = 0

        for d in range(n-1,-1,-1):
            for i in range(2**d):
                selected_str = to_bin(i, d)

                if not conflicts(G_str, selected_str):  # all selected nodes are not joined to current node
                    res = p * (1 + F[selected_str + "1"]) + (1-p) * F[selected_str + "0"]
                else:
                    res = F[selected_str + "0"]

                F[selected_str] = res

        greedy_val = F[""]
        
        
        ratio = opt_val / greedy_val if greedy_val > 0 else math.inf
        
        if first_iter:
            best_G = G_str
            comp_ratio = ratio
            first_iter = False
        elif ratio > comp_ratio:
            best_G = G_str
            comp_ratio = ratio
    
    return best_G, comp_ratio

## results

In [175]:
n = 4

In [176]:
comp_ratios = {}

for p in tqdm(np.linspace(0, 1, num=101)):
    comp_ratios[p] = compute_comp_ratio(n, p)

100%|█████████████████████████████████████████████████████████████| 101/101 [00:01<00:00, 95.14it/s]


In [177]:
p = min(comp_ratios, key=lambda string: comp_ratios.get(string)[1])

In [178]:
G_str, comp_ratio = comp_ratios[p]

In [179]:
print(f"with {p=:.2f}, graph {G_str} gives competitive ratio {comp_ratio:.6f}")

with p=0.67, graph 110100 gives competitive ratio 2.250056


In [184]:
print_graph(G_str)

(1, 2)
(1, 3)
(1, 4)


### ad hoc

In [182]:
compute_comp_ratio(7, 0.5)

('111110110011000110000', 3.6363636363636362)

In [185]:
print_graph('111110110011000110000')

(1, 2)
(1, 3)
(2, 3)
(1, 4)
(2, 4)
(1, 5)
(2, 5)
(1, 6)
(2, 6)
(1, 7)
(2, 7)
